In [1]:
import pymongo
import pandas as pd
import datetime
import math
from dateutil.parser import parse as parseDate
import sys
#from fuzzysearch import find_near_matches
sys.path.append('/home/immichail/anaconda3/lib/python3.8/site-packages')

db = pymongo.MongoClient()['ltcLongevity']

In [6]:
users = pd.read_excel('./data/2ndIter/Пользователи.xlsx')

In [11]:
users = users.rename(columns = {
    'уникальный номер': 'userId', 
    'дата создание личного дела': 'dateCreated', 
    'Фамилия': 'surName', 
    'Имя': 'name', 
    'Отчетсво': 'thirdName', 
    'пол': 'gender', 
    'дата рождения': 'dateBirth', 
    'адрес проживания': 'address'
})

In [14]:
db.usersV2.insert_many(users.to_dict(orient = 'records'))

In [107]:
df = pd.read_excel('./data/Расписание занятий(1).xlsx')

In [108]:
dfData = df.to_dict(orient = 'records')

df.columns

Index(['уникальный номер', 'направление 1', 'направление 2', 'направление 3',
       'адрес площадки', 'Округ', 'район', 'расписание в активных периодах 1',
       'расписание в закрытых периодах 1', 'расписание в плановом периоде',
       'Дата начала', 'Дата окончания', 'день 1', 'Время начала 1',
       'Время окончания 1', 'Время перерыва в минутах 1', 'день 2',
       'Время начала 2', 'Время окончания 2', 'Время перерыва в минутах 2',
       'день 3', 'Время начала 3', 'Время окончания 3',
       'Время перерыва в минутах 3', 'день 4', 'Время начала 4',
       'Время окончания 4', 'Время перерыва в минутах 4', 'день 5',
       'Время начала 5', 'Время окончания 5', 'Время перерыва в минутах 5',
       'день 6', 'Время начала 6', 'Время окончания 6',
       'Время перерыва в минутах 6', 'день 7', 'Время начала 7',
       'Время окончания 7', 'Время перерыва в минутах 7'],
      dtype='object')

In [85]:
dfDataP = []

for item in dfData:
    itemP = {
        'groupId': item['уникальный номер'],
        'd1LevelName': item['направление 1'],
        'd2LevelName': item['направление 2'],
        'd3LevelName': item['направление 3'],
        'address': item['адрес площадки'],
        'district': item['Округ'],
        'area': item['район'],
        'scheduleActive': item['расписание в активных периодах 1'],
        'scheduleClosed': item['расписание в закрытых периодах 1'],
        'schedulePlanned': item['расписание в плановом периоде'],
        'dateStarted': item['Дата начала'].to_pydatetime(),
        'dateFinished': parseDate(item['Дата окончания'].replace('.012.', '.12.'))
    }
    
    dates = []

    notNaTimeStart = None
    notNaTimeEnd = None

    for i in range(1, 8):
        if isinstance(item['день %d'%i], str):
            dates.append({
                'day': item['день %d'%i],
                'timeStart': item['Время начала %d'%i],
                'timeEnd': item['Время окончания %d'%i],
                'timeBreak': item['Время перерыва в минутах %d'%i]
            })

            if (isinstance(item['Время начала %d'%i], datetime.time)):
                notNaTimeStart = item['Время начала %d'%i].isoformat()
            if (isinstance(item['Время окончания %d'%i], datetime.time)):
                notNaTimeEnd = item['Время окончания %d'%i].isoformat()

    for d in dates:
        if not(isinstance(d['timeStart'], str)):
            d['timeStart'] = notNaTimeStart
        if not(isinstance(d['timeEnd'], str)):
            d['timeEnd'] = notNaTimeEnd

    itemP['schedule'] = dates
    
    dfDataP.append(itemP)

In [87]:
db.schedule.insert_many(dfDataP)

In [90]:
df = pd.read_excel('./data/Справочник активностей(1).xlsx')

In [97]:
dfData = df.to_dict(orient = 'records')

Index(['Разметка: Для ума/ Для души / Для тела', 'id_level1', 'level1',
       'id_level2', 'level2', 'id_level3', 'leve3', 'Признак "Онлайн"',
       'Признак "Выдается диплом"', 'd_level1', 'd_level2', 'd_level3'],
      dtype='object')

In [101]:
dfDataP = []

for item in dfData:
    itemP = {
        'd0LevelName': item['Разметка: Для ума/ Для души / Для тела'],
        'd1LevelId': item['id_level1'],
        'd1LevelName': item['level1'],
        'd2LevelId': item['id_level2'],
        'd2LevelName': item['level2'],
        'd3LevelId': item['id_level3'],
        'd3LevelName': item['leve3'],
        'online': item['Признак "Онлайн"'] == 'Да',
        'certificate': item['Признак "Выдается диплом"'] == 'Да',
        'title': item['d_level1']
    }
    
    dfDataP.append(itemP)

In [123]:
def fuzzySearch(q, d3LevelNames: list):
    d3LevelNamesSearch = []

    for n in d3LevelNames:
        try:
            minDist = sum([
                sorted(find_near_matches(qq.lower(), n.lower(), max_l_dist = len(qq) - 1), key = lambda x: x.dist)[0].dist
                for qq in q.split()
            ])

            d3LevelNamesSearch.append({
                'name': n, 
                'minDist': minDist
            })
        except:
            pass
        
    return d3LevelNamesSearch

'ОНЛАЙН Английский язык'

In [249]:
def searchFilters(filters, limit = 10, offset = 0):

    filtersActivities = {}

    if ('d0LevelName' in filters)and(filters['d0LevelName'] is not None):
        filtersActivities['d0LevelName'] = {'$in': filters['d0LevelName']}

    if ('d1LevelName' in filters)and(filters['d1LevelName'] is not None):
        filtersActivities['d1LevelName'] = {'$in': filters['d1evelName']}

    if ('d2LevelName' in filters)and(filters['d2LevelName'] is not None):
        filtersActivities['d2LevelName'] = {'$in': filters['d2LevelName']}

    if ('online' in filters)and(filters['online'] is not None):
        filtersActivities['online'] = filters['online']

    if ('certificate' in filters)and(filters['certificate'] is not None):
        filtersActivities['certificate'] = filters['certificate']

    if len(filtersActivities) > 0:
        d3LevelNames = set([i['d3LevelName'] for i in db.activities.find(filtersActivities, {'d3LevelName': 1})])
    else:
        d3LevelNames = None

    if ('q' in filters)and(filters['q'] is not None):
        if d3LevelNames is None:
            d3LevelNames = set([i['d3LevelName'] for i in db.activities.find({}, {'d3LevelName': 1})])

        d3LevelNamesRelevance = fuzzySearch(filters['q'], d3LevelNames)

        d3LevelNamesRelevance = {i['name']: i['minDist'] for i in d3LevelNamesRelevance}
    else:
        d3LevelNamesRelevance = None

    filtersSchedule = {}

    if ('days' in filters)and(filters['days'] is not None):
        filtersSchedule['schedule.day'] = {'$in': filters['days']}

    if ('district' in filters)and(filters['district'] is not None):
        filtersSchedule['district'] = {'$in': filters['district']}

    if ('area' in filters)and(filters['area'] is not None):
        filtersSchedule['area'] = {'$in': filters['area']}

    if d3LevelNames is not None:
        filtersSchedule['d3LevelName'] = {'$in': list(d3LevelNames)}

    res = list(db.schedule.find(filtersSchedule))

    if d3LevelNamesRelevance is not None:
        res = [r for r in res if r['d3LevelName'] in d3LevelNamesRelevance]
        res = sorted(res, key = lambda x: d3LevelNamesRelevance[x['d3LevelName']])

    res = res[offset:offset + limit]  

    return res

In [252]:
d0LevelName = ['Для ума']

filters = {
    'q': None,
    'd0LevelName': ['Для ума'],
    'd1LevelName': None,
    'd2LevelName': None,
    'online': None,
    'certificate': None,
    'days': None,
    'dates': None,
    'district': None,
    'area': None
}

limit = 10
offset = 0

searchFilters({
    'q': 'англ',
    'd0LevelName': ['Для ума'],
    'online': True,
    'days': ['Сб']
})

[{'_id': ObjectId('6470adcb0ddba6d6eda37093'),
  'groupId': 801354377,
  'd1LevelName': 'Образование',
  'd2LevelName': 'ОНЛАЙН Английский язык',
  'd3LevelName': 'ОНЛАЙН Английский язык',
  'address': 'г. Москва, Новокосинская улица, дом 13, корпус 1',
  'district': 'Восточный административный округ',
  'area': 'муниципальный округ Новокосино',
  'scheduleActive': nan,
  'scheduleClosed': 'c 16.01.2023 по 31.03.2023, Сб. 10:00-12:00, без перерыва',
  'schedulePlanned': nan,
  'dateStarted': datetime.datetime(2023, 1, 1, 0, 0),
  'dateFinished': datetime.datetime(2023, 12, 31, 0, 0),
  'schedule': [{'day': 'Сб',
    'timeStart': '10:00:00',
    'timeEnd': '10:00:00',
    'timeBreak': 0}]},
 {'_id': ObjectId('6470adcb0ddba6d6eda371b1'),
  'groupId': 801354396,
  'd1LevelName': 'Образование',
  'd2LevelName': 'ОНЛАЙН Английский язык',
  'd3LevelName': 'ОНЛАЙН Английский язык',
  'address': 'город Москва, Открытое шоссе, дом 24, корпус 27',
  'district': 'Восточный административный округ'

In [1]:
def getCategories(param: str):
    if param in ['d0LevelName', 'd1LevelName', 'd2LevelName']:
        return db.activities.distinct(param)
    if param in ['area', 'district']:
        return sorted([i.strip() for i in db.schedule.distinct(param) if (isinstance(i, str))and(i.strip() != '')])
    if param in ['online', 'certificate']:
        return ['Да'|"Нет"]
    if param in days:
        return ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']

In [5]:
import pandas as pd

df = pd.read_csv('./data/groups.csv')
dfData = df.to_dict(orient = 'records')

In [22]:
dfDataP = []

for item in dfData:
    itemP = {
        'groupId': item['уникальный номер'],
        'd1LevelName': item['направление 1'],
        'd2LevelName': item['направление 2'],
        'd3LevelName': item['направление 3'],
        'address': item['адрес площадки'],
        'district': item['округ площадки'],
        'area': item['район площадки'],
        'scheduleActive': item['расписание в активных периодах'],
        'scheduleClosed': item['расписание в закрытых периодах'],
        'schedulePlanned': item['расписание в плановом периоде']
    }
    
    dfDataP.append(itemP)

In [28]:
import json

survey = json.load(open('./data/index.json', 'r'))

db.vars.insert_one({
    'id': 'survey',
    'value': survey
})

In [32]:
db.schedule.find_one()

{'_id': ObjectId('6470adcb0ddba6d6eda36e97'),
 'groupId': 801357270,
 'd1LevelName': 'Физическая активность',
 'd2LevelName': 'ОФП',
 'd3LevelName': 'ОФП',
 'address': 'город Москва, Саратовская улица, дом 16, корпус 2',
 'district': 'Юго-Восточный административный округ',
 'area': 'муниципальный округ Текстильщики',
 'scheduleActive': nan,
 'scheduleClosed': 'c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:10, без перерыва',
 'schedulePlanned': nan,
 'dateStarted': datetime.datetime(2023, 1, 1, 0, 0),
 'dateFinished': datetime.datetime(2023, 12, 31, 0, 0),
 'schedule': [{'day': 'Пн',
   'timeStart': '19:10:00',
   'timeEnd': '19:10:00',
   'timeBreak': 0},
  {'day': 'Ср',
   'timeStart': '19:10:00',
   'timeEnd': '19:10:00',
   'timeBreak': 0}]}

In [46]:
from tqdm import tqdm

total = db.activities.count_documents({})

pbar = tqdm(total = total)

for item in db.activities.find({}):
    db.schedule.update_many({
        'd1LevelName': item['d1LevelName'],
        'd2LevelName': item['d2LevelName'],
        'd3LevelName': item['d3LevelName']
    }, {
        '$set': {
            'd0LevelName': item['d0LevelName'],
            'd1LevelId': item['d1LevelId'],
            'd2LevelId': item['d2LevelId'],
            'd3LevelId': item['d3LevelId']
        }
    })
    
    pbar.update(1)

100%|█████████▉| 896/899 [00:16<00:00, 45.21it/s]

In [51]:
df = pd.read_csv('./data/attend.csv')

In [54]:
dfData = df.to_dict(orient = 'records')

In [63]:
# 'workId': workId,
# 'groupId': groupId,
# 'userId': userId,
# 'd2LevelId': d2LevelId,
# 'd3LevelId': d3LevelId,
# 'online': onlineId,
# 'date': date

from datetime import time as datetimeTime
import logging

dfDataP = []

for item in dfData:
    
    try:
        itemP = {
            'workId': item['уникальный номер занятия'],
            'groupId': item['уникальный номер группы'],
            'userId': item['уникальный номер участника'],
            'd2LevelName': item['направление 2'],
            'd3LevelName': item['направление 3'],
            'online': item['онлайн/офлайн'] == 'Да',
            'date': parseDate(item['дата занятия']),
            'timeStarted': datetimeTime(
                hour = int(item['время начала занятия'].split(':')[0]),
                minute = int(item['время начала занятия'].split(':')[0])
            ),
            'timeEnded': datetimeTime(
                hour = int(item['время окончания занятия'].split(':')[0]),
                minute = int(item['время окончания занятия'].split(':')[0])
            )
        }

        dfDataP.append(itemP)
    except:
        logging.exception('err')

In [69]:
for item in dfDataP:
    item['timeStarted'] = item['timeStarted'].strftime('%H:%M')
    item['timeEnded'] = item['timeEnded'].strftime('%H:%M')

In [70]:
db.attends.insert_many(dfDataP)

In [80]:
total = db.activities.count_documents({})

pbar = tqdm(total = total)

for item in db.activities.find({}):
    db.attends.update_many({
        'd2LevelName': item['d2LevelName'],
        'd3LevelName': item['d3LevelName']
    }, {
        '$set': {
            'd0LevelName': item['d0LevelName'],
            'd1LevelId': item['d1LevelId'],
            'd2LevelId': item['d2LevelId'],
            'd3LevelId': item['d3LevelId']
        }
    })
    
    pbar.update(1)



  1%|          | 9/899 [00:46<1:16:09,  5.13s/it]


  0%|          | 1/899 [00:02<36:46,  2.46s/it]

  0%|          | 2/899 [00:04<36:26,  2.44s/it]

  0%|          | 3/899 [00:07<36:28,  2.44s/it]

  0%|          | 4/899 [00:09<36:21,  2.44s/it]

  1%|          | 5/899 [00:12<36:15,  2.43s/it]

  1%|          | 6/899 [00:14<36:20,  2.44s/it]

  1%|          | 7/899 [00:17<36:09,  2.43s/it]

  1%|          | 8/899 [00:19<36:15,  2.44s/it]

  1%|          | 9/899 [00:22<37:53,  2.55s/it]

  1%|          | 10/899 [00:24<37:31,  2.53s/it]

  1%|          | 11/899 [00:27<37:13,  2.52s/it]

  1%|▏         | 12/899 [00:29<37:03,  2.51s/it]

  1%|▏         | 13/899 [00:32<36:55,  2.50s/it]

  2%|▏         | 14/899 [00:34<36:47,  2.49s/it]

  2%|▏         | 15/899 [00:37<36:40,  2.49s/it]

  2%|▏         | 16/899 [00:39<36:37,  2.49s/it]

  2%|▏         | 17/899 [00:42<36:51,  2.51s/it]

  2%|▏         | 18/899 [00:44<36:41,  2.50s/it]

  2%|▏         | 19/899 [00:47<36:27,  2.49s/it]

  2%|

 32%|███▏      | 288/899 [12:47<26:10,  2.57s/it]

 32%|███▏      | 289/899 [12:50<26:18,  2.59s/it]

 32%|███▏      | 290/899 [12:52<26:13,  2.58s/it]

 32%|███▏      | 291/899 [12:55<26:09,  2.58s/it]

 32%|███▏      | 292/899 [12:57<26:10,  2.59s/it]

 33%|███▎      | 293/899 [13:00<26:08,  2.59s/it]

 33%|███▎      | 294/899 [13:03<26:01,  2.58s/it]

 33%|███▎      | 295/899 [13:05<25:57,  2.58s/it]

 33%|███▎      | 296/899 [13:08<25:47,  2.57s/it]

 33%|███▎      | 297/899 [13:10<25:44,  2.56s/it]

 33%|███▎      | 298/899 [13:13<25:37,  2.56s/it]

 33%|███▎      | 299/899 [13:15<25:38,  2.56s/it]

 33%|███▎      | 300/899 [13:18<25:31,  2.56s/it]

 33%|███▎      | 301/899 [13:20<25:29,  2.56s/it]

 34%|███▎      | 302/899 [13:23<25:27,  2.56s/it]

 34%|███▎      | 303/899 [13:26<25:32,  2.57s/it]

 34%|███▍      | 304/899 [13:28<25:36,  2.58s/it]

 34%|███▍      | 305/899 [13:31<25:33,  2.58s/it]

 34%|███▍      | 306/899 [13:33<25:32,  2.58s/it]

 34%|███▍      | 307/899 [13:36

 64%|██████▍   | 574/899 [25:03<13:56,  2.57s/it]

 64%|██████▍   | 575/899 [25:05<13:50,  2.56s/it]

 64%|██████▍   | 576/899 [25:08<13:47,  2.56s/it]

 64%|██████▍   | 577/899 [25:10<13:44,  2.56s/it]

 64%|██████▍   | 578/899 [25:13<13:42,  2.56s/it]

 64%|██████▍   | 579/899 [25:15<13:41,  2.57s/it]

 65%|██████▍   | 580/899 [25:18<13:38,  2.57s/it]

 65%|██████▍   | 581/899 [25:21<13:33,  2.56s/it]

 65%|██████▍   | 582/899 [25:23<13:28,  2.55s/it]

 65%|██████▍   | 583/899 [25:26<13:26,  2.55s/it]

 65%|██████▍   | 584/899 [25:28<13:26,  2.56s/it]

 65%|██████▌   | 585/899 [25:31<13:23,  2.56s/it]

 65%|██████▌   | 586/899 [25:33<13:22,  2.56s/it]

 65%|██████▌   | 587/899 [25:36<13:21,  2.57s/it]

 65%|██████▌   | 588/899 [25:38<13:20,  2.57s/it]

 66%|██████▌   | 589/899 [25:41<13:15,  2.57s/it]

 66%|██████▌   | 590/899 [25:44<13:12,  2.56s/it]

 66%|██████▌   | 591/899 [25:46<13:09,  2.56s/it]

 66%|██████▌   | 592/899 [25:49<13:05,  2.56s/it]

 66%|██████▌   | 593/899 [25:51

 96%|█████████▌| 860/899 [38:03<02:13,  3.41s/it]

 96%|█████████▌| 861/899 [38:06<02:11,  3.46s/it]

 96%|█████████▌| 862/899 [38:09<01:59,  3.24s/it]

 96%|█████████▌| 863/899 [38:11<01:50,  3.06s/it]

 96%|█████████▌| 864/899 [38:16<02:01,  3.47s/it]

 96%|█████████▌| 865/899 [38:19<01:50,  3.25s/it]

 96%|█████████▋| 866/899 [38:21<01:42,  3.10s/it]

 96%|█████████▋| 867/899 [38:27<01:59,  3.73s/it]

 97%|█████████▋| 868/899 [38:29<01:45,  3.40s/it]

 97%|█████████▋| 869/899 [38:32<01:35,  3.18s/it]

 97%|█████████▋| 870/899 [38:35<01:28,  3.05s/it]

 97%|█████████▋| 871/899 [38:37<01:21,  2.92s/it]

 97%|█████████▋| 872/899 [38:40<01:17,  2.86s/it]

 97%|█████████▋| 873/899 [38:44<01:23,  3.21s/it]

 97%|█████████▋| 874/899 [38:47<01:17,  3.10s/it]

 97%|█████████▋| 875/899 [38:50<01:11,  2.98s/it]

 97%|█████████▋| 876/899 [38:52<01:06,  2.89s/it]

 98%|█████████▊| 877/899 [38:55<01:03,  2.86s/it]

 98%|█████████▊| 878/899 [38:58<00:59,  2.82s/it]

 98%|█████████▊| 879/899 [39:00

In [23]:
db.schedule.update_many({}, {'$set': {'active': False}})

In [13]:
df = pd.read_excel('./data/Расписание занятий 28.05.xlsx')

In [26]:
df = df[['уникальный номер', 'Участвует в рекомендации']]

for groupId in df[df['Участвует в рекомендации'] == 'Да']['уникальный номер']:
    db.schedule.update_one({'groupId': int(groupId)}, {'$set': {'active': True}})

In [25]:
db.schedule.find_one({'active': True})

In [33]:
import json

var = json.load(open('./data/survey.json', 'r'))

db.vars.update_one({'id': 'survey'}, {'$set': {'value': var}})